<a href="https://colab.research.google.com/github/renowator/My_ML/blob/master/Sound_Recognition_Conv1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Upload the sample files to Google Colab and extract .zip**

In [33]:
from google.colab import files
uploaded = files.upload()


In [0]:
#!unzip Samples.zip

Load the files to a list and add corresponding label. *Skip problematic files*

In [1]:
import os
from scipy.io import wavfile
import numpy as np
data = []
labels = []
directory = './Samples/Kicks/'
!cd Samples/Kicks
for filename in os.listdir(directory):
  try:
    if filename.endswith(".wav") or filename.endswith(".WAV"):
        fs, new_data = wavfile.read(directory + filename)
        data.append(np.array(new_data))
        labels.append(0)
        continue
    else:
      continue
  except:
    print("Oops")
    continue
    
directory = './Samples/Snares/'
for filename in os.listdir(directory):
  try:
    if filename.endswith(".wav") or filename.endswith(".WAV"):
        fs, new_data = wavfile.read(directory + filename)
        data.append(np.array(new_data))
        labels.append(1)
        continue
    else:
        continue
  except:
    print("Oops")
    continue
directory = './Samples/HHT/'
for filename in os.listdir(directory):
  try:
    if filename.endswith(".wav") or filename.endswith(".WAV"):
        fs, new_data = wavfile.read(directory + filename)
        data.append(np.array(new_data))
        labels.append(2)
        continue
    else:
        continue
  except:
    print("Oops")
    continue
    
    
print(len(data), len(labels))

/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
Oops
466 466


Remove items with unusual shape

In [2]:

for i, item in enumerate(data):
  if len(item.shape) != 2:
    del data[i]
    del labels[i]
    print(len(data))
    print(len(labels))

465
465
464
464
463
463
462
462
461
461
460
460
459
459
458
458
457
457


Process items to give them same size (size of max file+220) and put into numpy array for further processing.

In [3]:
desired_shape = (np.max([np.max(item.shape) for item in data ]) + 220 , 2)
print(desired_shape)
processed_data = None
for i, item in enumerate(data):
  if (len(item.shape) == 2):
    result = np.zeros(desired_shape).astype(np.int16)
    result[:item.shape[0],:item.shape[1]] = item.astype(np.int16)
    result = result.reshape((1, desired_shape[0], desired_shape[1]))
    if processed_data is None:
      processed_data = result
    else:
      print("\r"+str( result.shape) + " ==> " + str(processed_data.shape), end="")
      processed_data = np.append(processed_data, result, axis=0)
  else:
    del labels[i]
print(processed_data.shape)

(1254620, 2)
(1, 1254620, 2) ==> (455, 1254620, 2)(456, 1254620, 2)


In [0]:

labels = np.array(labels).astype(np.int16)

Now we need to get processed_data to be in range of [-1;1] instead of current [-32000; 32000]

In [5]:
processed_data[200]

array([[   0,    0],
       [-451, -489],
       [  48,   39],
       ...,
       [   0,    0],
       [   0,    0],
       [   0,    0]], dtype=int16)

In [0]:
max_val = np.max(processed_data)
processed_data = processed_data.astype(np.float16)
processed_data = processed_data/max_val

In [7]:
processed_data[200]

array([[ 0.        ,  0.        ],
       [-0.01376385, -0.01492355],
       [ 0.00146489,  0.00119022],
       ...,
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]], dtype=float32)

In [8]:
from keras.utils import to_categorical

categorical_labels = to_categorical(labels, num_classes=3)

Using TensorFlow backend.


In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
## extra imports to set GPU options
import tensorflow as tf
from keras import backend as k
 
###################################
# TensorFlow wizardry
config = tf.ConfigProto()
 
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True
 
# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.95
config.inter_op_parallelism_threads=4
config.intra_op_parallelism_threads=4
# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))
###################################

**Create a NN model utilizing AveragePooling(for incoming samples); Temporal 1D convolutions and MaxPooling layers and finish with Fully Connected layer and Softmax output layer**

In [0]:
model = Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(desired_shape[0], desired_shape[1])))
model.add(tf.keras.layers.AveragePooling1D(22, 2))
model.add(tf.keras.layers.Conv1D(4, 3, 2, "valid", activation="relu"))
model.add(tf.keras.layers.MaxPooling1D(3, 2))
model.add(tf.keras.layers.Conv1D(16, 3, 2, "valid", activation="relu"))
model.add(tf.keras.layers.MaxPooling1D(3, 2))
model.add(tf.keras.layers.Conv1D(64, 3, 2, "valid", activation="relu"))
model.add(tf.keras.layers.MaxPooling1D(3, 2))
model.add(tf.keras.layers.Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [39]:
opt = tf.keras.optimizers.RMSprop(lr=0.002)

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
average_pooling1d_7 (Average (None, 627300, 2)         0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 313649, 4)         28        
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 156824, 4)         0         
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 78411, 16)         208       
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 39205, 16)         0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 19602, 64)         3136      
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 9800, 64)          0         
__________

In [40]:
# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(processed_data, categorical_labels, epochs=20, verbose=1, validation_split=0.2, batch_size= 16, shuffle=True, callbacks=callbacks_list)

Train on 364 samples, validate on 92 samples
Epoch 1/20
352/364 [============================>.] - ETA: 0s - loss: 0.8128 - acc: 0.6648
Epoch 00001: val_loss improved from inf to 2.67035, saving model to weights.best.hdf5
364/364 [==============================] - 5s 15ms/sample - loss: 0.8038 - acc: 0.6676 - val_loss: 2.6703 - val_acc: 0.0000e+00
Epoch 2/20
352/364 [============================>.] - ETA: 0s - loss: 0.3995 - acc: 0.8295
Epoch 00002: val_loss improved from 2.67035 to 2.11387, saving model to weights.best.hdf5
364/364 [==============================] - 4s 12ms/sample - loss: 0.3926 - acc: 0.8297 - val_loss: 2.1139 - val_acc: 0.0000e+00
Epoch 3/20
352/364 [============================>.] - ETA: 0s - loss: 0.3403 - acc: 0.8778
Epoch 00003: val_loss improved from 2.11387 to 0.69377, saving model to weights.best.hdf5
364/364 [==============================] - 4s 12ms/sample - loss: 0.3419 - acc: 0.8791 - val_loss: 0.6938 - val_acc: 0.8804
Epoch 4/20
352/364 [================

Load the weights with best validation performance and save our model.

In [0]:
model.load_weights("weights.best.hdf5")

In [0]:
model.save('sound_rec_model.h5')

In [0]:
from google.colab import files
#files.download('sound_rec_model.h5') 

Now we can test the Classification accuracy with other samples not used in training/validation

In [43]:
from google.colab import files
#uploaded = files.upload()


fs, data = wavfile.read('Phonotix Kick 1.wav')
test_data = np.array(data).astype(np.int16)
result = np.zeros(desired_shape).astype(np.float16)
result[:test_data.shape[0],:test_data.shape[1]] = test_data/max_val
test_data = result.reshape((1, desired_shape[0], desired_shape[1]))


/usr/local/lib/python3.6/dist-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [44]:
np.argmax(model.predict(test_data))

0

In [45]:
model.predict(processed_data[0].reshape(1, 1254620, 2))

array([[9.9944061e-01, 5.5938511e-04, 1.0140273e-09]], dtype=float32)

In [46]:
model.predict(processed_data[250].reshape(1, 1254620, 2))

array([[1.0671884e-04, 9.9989259e-01, 6.5943419e-07]], dtype=float32)

In [47]:
model.predict(processed_data[400].reshape(1, 1254620, 2))

array([[0.00523018, 0.08496662, 0.90980315]], dtype=float32)

In [0]:
type(test_data[0][0][0])

numpy.float16